In [1]:
import os
import pandas as pd
from modules.postgres_functions import probar_conexion_postgresql, create_sql_script, sql_query, df_to_sql

host = "localhost"
port = 5432
database = "kurtdb2"
user = "postgres"
password = "gneomar/99"

# Probar conexion
probar_conexion_postgresql(host, user, password, database, port)


Conexión exitosa a PostgreSQL


In [3]:
import pandas as pd
from faker import Faker
import random

# Inicializa Faker
fake = Faker()

# Función para generar datos falsos
def generar_datos_fake(num_rows):
    datos = []
    for _ in range(num_rows):
        id_ = _ + 1
        nombre = fake.first_name()
        apellido = fake.last_name()
        cantidad = random.randint(1, 100)
        monto = round(random.uniform(10.0, 1000.0), 2)
        
        datos.append({
            "id": id_,
            "nombre": nombre,
            "apellido": apellido,
            "cantidad": cantidad,
            "monto": monto
        })
    
    return pd.DataFrame(datos)

# Número de filas que deseas generar
num_filas = 10

# Crear el DataFrame
df = generar_datos_fake(num_filas)

# Mostrar el DataFrame
df


,id,nombre,apellido,cantidad,monto
0,1,Steven,Howard,92,794.27
1,2,Karen,Lane,4,975.12
2,3,Philip,Wilson,36,464.49
3,4,Allen,Huynh,53,751.65
4,5,Amy,Sullivan,19,20.88
5,6,Marie,Petersen,8,798.74
6,7,Anita,Moore,79,588.54
7,8,Tina,Graham,33,779.56
8,9,Natalie,Harris,24,562.69
9,10,Sheila,Lowe,73,419.25


In [ ]:
df_to_sql(host, password, database, user, port, df, 'ejemplo_tabla', 'append')

In [6]:
import os
import pandas as pd
from modules.postgres_functions import probar_conexion_postgresql, create_sql_script, sql_query, df_to_sql

host = "localhost"
port = 5434
database = "kurtdb"
user = "postgres"
password = "kurt2025"

# Probar conexion
probar_conexion_postgresql(host, user, password, database, port)


Conexión exitosa a PostgreSQL


In [7]:
import pandas as pd
import psycopg2
from psycopg2 import sql

def generate_transaction_script(df, table_name, error_table_name):
    """
    Genera un script SQL para insertar filas de un DataFrame en PostgreSQL con manejo de errores.
    
    Args:
        df (pd.DataFrame): DataFrame con los datos a insertar.
        table_name (str): Nombre de la tabla destino.
        error_table_name (str): Nombre de la tabla para registrar errores.
    
    Returns:
        str: Script SQL.
    """
    values = []
    for row in df.itertuples(index=False):
        row_values = ", ".join(
            f"'{x}'" if isinstance(x, str) else "NULL" if pd.isna(x) else str(x)
            for x in row
        )
        values.append(f"({row_values})")
    
    values_sql = ",\n            ".join(values)

    script = f"""
DO $$
DECLARE
    fila RECORD;
BEGIN
    -- Iterar sobre las filas a insertar
    FOR fila IN
        SELECT * FROM (VALUES
            {values_sql}
        ) AS datos(dni, nombre, apellido, cantidad, monto)
    LOOP
        BEGIN
            -- Intentar insertar la fila
            INSERT INTO {table_name} (dni, nombre, apellido, cantidad, monto)
            VALUES (fila.dni, fila.nombre, fila.apellido, fila.cantidad, fila.monto);
        EXCEPTION
            WHEN OTHERS THEN
                -- Registrar el error en la tabla de errores
                INSERT INTO {error_table_name} (error_message, data)
                VALUES (SQLERRM, row_to_json(fila));
        END;
    END LOOP;
END $$;
"""
    return script

In [8]:
# Ejemplo de uso
data = {
    "dni": [123456, 654321, 123456, 789123],
    "nombre": ["Juan", "Maria", "Pedro", "Ana"],
    "apellido": ["Perez", "Lopez", "Gomez", "Martinez"],
    "cantidad": [10, 5, 8, None],
    "monto": [100.50, 200.75, 300.00, 150.00],
}

# Crear DataFrame
df = pd.DataFrame(data)

In [9]:
# Generar script SQL
script = generate_transaction_script(df, "ejemplo_tabla", "errores_ejemplo_tabla")
print(script)


DO $$
DECLARE
    fila RECORD;
BEGIN
    -- Iterar sobre las filas a insertar
    FOR fila IN
        SELECT * FROM (VALUES
            (123456, 'Juan', 'Perez', 10.0, 100.5),
            (654321, 'Maria', 'Lopez', 5.0, 200.75),
            (123456, 'Pedro', 'Gomez', 8.0, 300.0),
            (789123, 'Ana', 'Martinez', NULL, 150.0)
        ) AS datos(dni, nombre, apellido, cantidad, monto)
    LOOP
        BEGIN
            -- Intentar insertar la fila
            INSERT INTO ejemplo_tabla (dni, nombre, apellido, cantidad, monto)
            VALUES (fila.dni, fila.nombre, fila.apellido, fila.cantidad, fila.monto);
        EXCEPTION
            WHEN OTHERS THEN
                -- Registrar el error en la tabla de errores
                INSERT INTO errores_ejemplo_tabla (error_message, data)
                VALUES (SQLERRM, row_to_json(fila));
        END;
    END LOOP;
END $$;



In [18]:
sql_query(host, password, database, user, port, script, values=None, fetch=None)

Error executing query: no results to fetch


In [ ]:

# Ejecución del script en PostgreSQL
def execute_script(connection_params, script):
    """
    Ejecuta un script SQL en PostgreSQL.
    
    Args:
        connection_params (dict): Parámetros de conexión a la base de datos.
        script (str): Script SQL a ejecutar.
    """
    try:
        with psycopg2.connect(**connection_params) as conn:
            with conn.cursor() as cur:
                cur.execute(script)
                conn.commit()
                print("Script ejecutado correctamente.")
    except Exception as e:
        print(f"Error al ejecutar el script: {e}")

# Parámetros de conexión (ajustar según tu entorno)
connection_params = {
    "dbname": "tu_base_de_datos",
    "user": "tu_usuario",
    "password": "tu_contraseña",
    "host": "localhost",
    "port": 5432,
}

# Ejecutar el script
execute_script(connection_params, script)


In [12]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import json

# Crear DataFrame con datos
data = {
    "dni": [123456, 654321, 123456, 789123],
    "nombre": ["Juan", "Maria", "Pedro", "Ana"],
    "apellido": ["Perez", "Lopez", "Gomez", "Martinez"],
    "cantidad": [10, 5, 8, None],
    "monto": [100.50, 200.75, 300.00, 150.00],
}
df = pd.DataFrame(data)

# Conexión a la base de datos
connection_params = {
    "dbname": "kurtdb",
    "user": "postgres",
    "password": "kurt2025",
    "host": "localhost",
    "port": 5434,
}

try:
    with psycopg2.connect(**connection_params) as conn:
        with conn.cursor() as cur:
            # Comenzar la transacción
            cur.execute("BEGIN;")
            
            for _, row in df.iterrows():
                try:
                    # Construir la consulta de inserción
                    insert_query = sql.SQL("""
                        INSERT INTO ejemplo_tabla (dni, nombre, apellido, cantidad, monto)
                        VALUES (%s, %s, %s, %s, %s)
                    """)
                    # Ejecutar la consulta con valores de la fila
                    cur.execute(insert_query, (row['dni'], row['nombre'], row['apellido'], row['cantidad'], row['monto']))
                except psycopg2.Error as e:
                    # Manejo de errores: registrar en la tabla de errores
                    error_query = sql.SQL("""
                        INSERT INTO errores_ejemplo_tabla (error_message, data)
                        VALUES (%s, %s)
                    """)
                    error_data = {
                        "dni": row['dni'],
                        "nombre": row['nombre'],
                        "apellido": row['apellido'],
                        "cantidad": row['cantidad'],
                        "monto": row['monto'],
                    }
                    cur.execute(error_query, (str(e), json.dumps(error_data)))
            
            # Confirmar la transacción
            conn.commit()
            print("Transacción completada exitosamente.")
except Exception as e:
    print(f"Error al procesar la transacción: {e}")


Error al procesar la transacción: current transaction is aborted, commands ignored until end of transaction block



In [19]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import json

def generate_transaction_script(df, table_name, error_table_name):
    """
    Genera un script SQL dinámico para insertar filas de un DataFrame en PostgreSQL con manejo de errores.
    
    Args:
        df (pd.DataFrame): DataFrame con los datos a insertar.
        table_name (str): Nombre de la tabla destino.
        error_table_name (str): Nombre de la tabla para registrar errores.
    
    Returns:
        str: Script SQL.
    """
    # Obtener columnas del DataFrame
    columns = df.columns.tolist()
    
    # Generar la lista de columnas en formato SQL
    columns_sql = ", ".join(columns)
    
    # Generar las filas en formato SQL VALUES
    values = []
    for row in df.itertuples(index=False):
        row_values = ", ".join(
            f"'{x}'" if isinstance(x, str) else "NULL" if pd.isna(x) else str(x)
            for x in row
        )
        values.append(f"({row_values})")
    
    values_sql = ",\n            ".join(values)

    # Generar el script SQL
    script = f"""
DO $$
DECLARE
    fila RECORD;
BEGIN
    -- Iterar sobre las filas a insertar
    FOR fila IN
        SELECT * FROM (VALUES
            {values_sql}
        ) AS datos({', '.join(columns)})
    LOOP
        BEGIN
            -- Intentar insertar la fila
            INSERT INTO {table_name} ({columns_sql})
            VALUES ({', '.join(f'fila.{col}' for col in columns)});
        EXCEPTION
            WHEN OTHERS THEN
                -- Registrar el error en la tabla de errores
                INSERT INTO {error_table_name} (error_message, data)
                VALUES (SQLERRM, row_to_json(fila));
        END;
    END LOOP;
END $$;
"""
    return script


In [36]:
# DataFrame de ejemplo con diferentes columnas
data = {
    "dni": [1, 2, 3, 4],
    "nombre": ["Juan", "Maria", "Pedro", "Ana"],
    "apellido": ["Perez", "Lopez", "Gomez", "Martinez"],
    "cantidad": [30, 25, 40, 35],
    "monto": [1000.50, 2000.75, 1500.00, None],
}
df = pd.DataFrame(data)

# Generar el script SQL dinámico
table_name = "ejemplo_tabla"
error_table_name = "errores_ejemplo_tabla"
script = generate_transaction_script(df, table_name, error_table_name)

# Imprimir el script
print(script)



DO $$
DECLARE
    fila RECORD;
BEGIN
    -- Iterar sobre las filas a insertar
    FOR fila IN
        SELECT * FROM (VALUES
            (1, 'Juan', 'Perez', 30, 1000.5),
            (2, 'Maria', 'Lopez', 25, 2000.75),
            (3, 'Pedro', 'Gomez', 40, 1500.0),
            (4, 'Ana', 'Martinez', 35, NULL)
        ) AS datos(dni, nombre, apellido, cantidad, monto)
    LOOP
        BEGIN
            -- Intentar insertar la fila
            INSERT INTO ejemplo_tabla (dni, nombre, apellido, cantidad, monto)
            VALUES (fila.dni, fila.nombre, fila.apellido, fila.cantidad, fila.monto);
        EXCEPTION
            WHEN OTHERS THEN
                -- Registrar el error en la tabla de errores
                INSERT INTO errores_ejemplo_tabla (error_message, data)
                VALUES (SQLERRM, row_to_json(fila));
        END;
    END LOOP;
END $$;



In [43]:
sql_query(host, password, database, user, port, script, values=None, fetch=None)

Error executing query: no results to fetch


In [41]:
import pandas as pd
from faker import Faker
import random

# Inicializa Faker
fake = Faker()

# Función para generar datos falsos
def generar_datos_fake(num_rows):
    datos = []
    for _ in range(num_rows):
        id_ = random.randint(1,20)
        nombre = fake.first_name()
        apellido = fake.last_name()
        cantidad = random.randint(1, 100)
        monto = round(random.uniform(10.0, 1000.0), 2)
        
        datos.append({
            "dni": id_,
            "nombre": nombre,
            "apellido": apellido,
            "cantidad": cantidad,
            "monto": monto
        })
    
    return pd.DataFrame(datos)

# Número de filas que deseas generar
num_filas = 10

# Crear el DataFrame
df2 = generar_datos_fake(num_filas)

script = generate_transaction_script(df2, table_name, error_table_name)

# Imprimir el script
print(script)



DO $$
DECLARE
    fila RECORD;
BEGIN
    -- Iterar sobre las filas a insertar
    FOR fila IN
        SELECT * FROM (VALUES
            (16, 'Stephen', 'Turner', 24, 464.58),
            (4, 'William', 'Vazquez', 11, 78.27),
            (6, 'Maria', 'Frey', 6, 992.0),
            (9, 'Daniel', 'Mason', 5, 248.36),
            (3, 'Charles', 'Chavez', 39, 43.18),
            (12, 'Beth', 'Huber', 4, 352.18),
            (18, 'Robert', 'Williams', 25, 578.65),
            (1, 'Justin', 'Ford', 89, 481.7),
            (19, 'Teresa', 'Martinez', 84, 809.91),
            (19, 'William', 'Carter', 26, 657.56)
        ) AS datos(dni, nombre, apellido, cantidad, monto)
    LOOP
        BEGIN
            -- Intentar insertar la fila
            INSERT INTO ejemplo_tabla (dni, nombre, apellido, cantidad, monto)
            VALUES (fila.dni, fila.nombre, fila.apellido, fila.cantidad, fila.monto);
        EXCEPTION
            WHEN OTHERS THEN
                -- Registrar el error en la tabla de 

In [42]:
df2

,dni,nombre,apellido,cantidad,monto
0,16,Stephen,Turner,24,464.58
1,4,William,Vazquez,11,78.27
2,6,Maria,Frey,6,992.00
3,9,Daniel,Mason,5,248.36
4,3,Charles,Chavez,39,43.18
5,12,Beth,Huber,4,352.18
6,18,Robert,Williams,25,578.65
7,1,Justin,Ford,89,481.70
8,19,Teresa,Martinez,84,809.91
9,19,William,Carter,26,657.56
